## Paquetes

In [1]:
import numpy as np
import pandas as pd
from itertools import chain
from tqdm import tqdm as tq

## Importación de matrices

In [5]:
MIP = np.genfromtxt('MIP_REGIONAL.csv', delimiter=';', dtype='str'); MIP[0,0] = MIP[0,0].replace("ï»¿", "")
MIP = np.array([[float(valor.replace(',', '.')) for valor in fila] for fila in MIP])

X = np.genfromtxt('X.csv', delimiter=';', dtype='str'); X[0] = X[0].replace("ï»¿", "")
X = np.array([float(valor.replace(',', '.')) for valor in X])

Y = np.genfromtxt('Y.csv', delimiter=';', dtype='str'); Y[0] = Y[0].replace("ï»¿", "")
Y = np.array([float(valor.replace(',', '.')) for valor in Y])


## Verificación metodologica

In [6]:
Z = np.divide(MIP, X, out=np.zeros_like(MIP), where=X!=0)
I = np.eye(MIP.shape[0])
IZ = I-Z
B = np.linalg.inv(IZ)

## Herramienta

### Indices y diccionarios

#### Sectores

In [7]:
sectores = pd.read_excel("DICCIONARIO.xlsx", sheet_name="Sectors")

Repetición de 54 sectores 33 veces

In [8]:
sectores_ix = []
for i in range(0,33):
    sectores_ix.append(list(range(0,54)))

sectores_ix_chained =  list(chain(*sectores_ix))

Repetición de 54 sectores 33 veces (pero con los indices de cada sector en la matrix completa)

In [9]:
sectores_region_ix = []

for i in range(0,33):
    m = 54*i
    list_sr = sectores_ix[i]
    list_sr = [k+m for k in list_sr]
    sectores_region_ix.append(list_sr)

### Regiones

In [10]:
regiones = pd.read_excel("DICCIONARIO.xlsx", sheet_name="Regions")

In [11]:
regiones_ix = [[r]*54 for r in list(sectores.ID_SECTOR)] #LISTA DE 33 LISTAS, CADA LISTA INTERNA DE LEN 54 (SECTORES) CON EL ID REPETIDO DE CADA REGION
regiones_ix_chained = list(chain(*regiones_ix)) #LO ANTERIOR EN UNA SOLA LISTA

### Sectores choque

Indices generales de sectores (de 1 a 54)

In [12]:
agro = [1,2,3,4,5]
shock = [6,7,23,34]
food = [10,11,12,13,14,15,16,17]
tour = [45]


Indices de sectores-región list of list y chained

In [13]:
# def multiplicar_lista(lista, rango_max):
    
#     lista_v2 = [x-1 for x in lista]
#     lista_resultante = [lista_v2]
    
#     for i in range(0, rango_max + 1):
#         k = 53*i
#         sublista = [(elemento + k) for elemento in lista]
#         # print(sublista)
#         lista_resultante.append(sublista)
    
#     return lista_resultante

In [14]:
#ESTOS SON LOS QUE SIRVEN PARA LOS CHOQUES (SON LOS INDICES DE ESE SECTOR EN ESPECIFICO PARA CADA UNO DE LOS DEPARTAMENTOS)
# agro_ix = multiplicar_lista(agro,33)
# shock_ix = multiplicar_lista(shock,33)
# food_ix = multiplicar_lista(food,33)
# tour_ix = multiplicar_lista(tour,33)

In [15]:
# agro_ix_chained = list(chain(*agro_ix))
# shock_ix_chained = list(chain(*shock_ix))
# food_ix_chained =list(chain(*food_ix))
# tour_ix_chained =  list(chain(*tour_ix))

### Descriptivas

Peso de la región en el PIB país

In [16]:
GDP_REGION = pd.DataFrame(zip(regiones_ix_chained,Y), columns = ["ID_REGION", "GDP"])
GDP_REGION = GDP_REGION.groupby("ID_REGION").sum().reset_index()
GDP_REGION["GDP SHARE"] = (GDP_REGION["GDP"]/Y.sum())*100
GDP_REGION = pd.merge(regiones, GDP_REGION, "left", "ID_REGION")
GDP_REGION

,ID,ID_REGION,Dane Code,Region,Departamento,GDP,GDP SHARE
0,1,0,5,R1,Antioquia,121629.454956,14.193691
1,2,1,8,R2,Atlántico,39920.280152,4.658544
2,3,2,11,R3,Bogotá D.C.,206770.522276,24.129327
3,4,3,13,R4,Bolívar,29005.964253,3.384885
4,5,4,15,R5,Boyacá,21090.033369,2.461126
5,6,5,17,R6,Caldas,14441.421069,1.685258
6,7,6,18,R7,Caquetá,4491.769505,0.524172
7,8,7,19,R8,Cauca,14829.700627,1.730569
8,9,8,20,R9,Cesar,17632.148178,2.057604
9,10,9,23,R10,Córdoba,15554.590529,1.815161


Peso del sector en el PIB país

In [17]:
GDP_SECTOR = pd.DataFrame(zip(sectores_ix_chained,Y), columns = ["ID_SECTOR", "GDP"])
GDP_SECTOR = GDP_SECTOR.groupby("ID_SECTOR").sum().reset_index()
GDP_SECTOR["GDP SHARE"] = (GDP_SECTOR["GDP"]/Y.sum())*100
GDP_SECTOR = pd.merge(sectores, GDP_SECTOR, "left", "ID_SECTOR")
GDP_SECTOR

,ID,ID_SECTOR,Sector,Sector_i,Sector_CIIU,CIIU Rev. 4 A.C.,GDP,GDP SHARE
0,1,0,S1,A,Agricultura y actividades de servicios conexas,A0101,21548.465974,2.514623
1,2,1,S2,A,Cultivo permanente de café,003,1552.793308,0.181205
2,3,2,S3,A,"Ganadería, caza y actividades de servicios con...",A0102,5274.083409,0.615465
3,4,3,S4,A,Silvicultura y extracción de madera,A02,1095.079417,0.127792
4,5,4,S5,A,Pesca y acuicultura,A03,1634.538079,0.190744
5,6,5,S6,HC,Extracción de carbón de piedra y lignito,017,11747.481183,1.370886
6,7,6,S7,HC,Extracción de petróleo crudo y gas natural y a...,018 021 022,40825.302326,4.764156
7,8,7,S8,O,Extracción de minerales metalíferos,019,243.491231,0.028414
8,9,8,S9,O,Extracción de otras minas y canteras,020,434.321046,0.050684
9,10,9,S10,FP,Procesamiento y conservación de carne y produc...,023 024 025,19503.898782,2.276030


In [19]:
GDP_SECTOR.groupby(["Sector_i"]).sum()

<ipython-input-19-2502a0faec27>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  GDP_SECTOR.groupby(["Sector_i"]).sum()


,ID,ID_SECTOR,GDP,GDP SHARE
Sector_i,,,,
A,15,10,31104.960186,3.629830
FP,108,100,59921.716269,6.992635
HC,70,66,68020.438092,7.937724
O,1247,1211,644126.455018,75.167088
T,45,44,53752.609064,6.272723


Peso del sector en el PIB región, PIB sector, PIB nacional

In [20]:
RS_DGP = pd.DataFrame(zip(regiones_ix_chained, sectores_ix_chained, Y), columns = ["ID_REGION", "ID_SECTOR", "GDP"])
GDP_SECTORM = GDP_SECTOR[["ID_SECTOR", "GDP"]]; GDP_SECTORM.columns = ["ID_SECTOR", "GDP_SECTOR"]; RS_DGP = RS_DGP.merge(GDP_SECTORM, "left", "ID_SECTOR")
GDP_REGIONM = GDP_REGION[["ID_REGION", "GDP"]]; GDP_REGIONM.columns = ["ID_REGION", "GDP_REGION"]; RS_DGP = RS_DGP.merge(GDP_REGIONM, "left", "ID_REGION")
RS_DGP["GDP_NAL"] = Y.sum()

RS_DGP["GDP_SECTOR_SHARE"] = RS_DGP["GDP"]/RS_DGP["GDP_SECTOR"]*100
RS_DGP["GDP_REGION_SHARE"] = RS_DGP["GDP"]/RS_DGP["GDP_REGION"]*100
RS_DGP["GDP_NAL_SHARE"]    = RS_DGP["GDP"]/RS_DGP["GDP_NAL"]*100

RS_DGP = RS_DGP.merge(regiones[["ID_REGION", "Departamento"]], "left", "ID_REGION")
RS_DGP = RS_DGP.merge(sectores[["ID_SECTOR", "Sector_i", "Sector_CIIU"]], "left", "ID_SECTOR")

In [21]:
RS_GDP_I = RS_DGP.groupby(["ID_REGION","Sector_i"]).sum().reset_index()
RS_GDP_I = RS_GDP_I.drop(columns=["GDP_SECTOR", "GDP_SECTOR_SHARE"])

GDP_SECTOR_I = GDP_SECTOR.groupby(["Sector_i"]).sum()["GDP"].reset_index(); GDP_SECTOR_I.columns = ["Sector_i", "GDP_SECTOR_I"]

RS_GDP_I = RS_GDP_I.merge(GDP_SECTOR_I,  "left", "Sector_i")
RS_GDP_I["GDP_SECTOR_I_SHARE"] = RS_GDP_I["GDP"]/RS_GDP_I["GDP_SECTOR_I"]*100

<ipython-input-21-f787e2c37fff>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  RS_GDP_I = RS_DGP.groupby(["ID_REGION","Sector_i"]).sum().reset_index()
<ipython-input-21-f787e2c37fff>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  GDP_SECTOR_I = GDP_SECTOR.groupby(["Sector_i"]).sum()["GDP"].reset_index(); GDP_SECTOR_I.columns = ["Sector_i", "GDP_SECTOR_I"]


## Choques

### Choque inicial a HC

In [48]:
HC_SHOCK = RS_DGP[RS_DGP.Sector_i=="HC"].index

In [49]:
s = 0.5
choque  = 1-s

YC = Y.copy()
ZC = Z.copy()

for i in HC_SHOCK:
    ZC[i,:] = ZC[i,:]*choque
    ZC[:,i] = ZC[:,i]*choque

    YC[i] = YC[i]*choque

# ZC[np.ix_(prueba, prueba)] *= choque
IC = np.eye(MIP.shape[0])
IZC = I-ZC
BC = np.linalg.inv(IZC)
XC = BC@YC

Caida X

In [50]:
CAIDA_X = (XC.sum()-X.sum())
print(f"MM: {CAIDA_X}")
print(f"%: {CAIDA_X/X.sum()}")

MM: -108095.61211408558
%: -0.07501732346788595


Caida Regional

In [51]:
X_REGION = pd.DataFrame(zip(regiones_ix_chained,X, XC), columns = ["ID_REGION", "X", "X'"])
X_REGION = X_REGION.groupby("ID_REGION").sum().reset_index()
X_REGION["X'-X"] = X_REGION["X'"]-X_REGION["X"]
X_REGION["X'-X/X"] = (X_REGION["X'-X"]/X_REGION["X"])*100
X_REGION = pd.merge(regiones, X_REGION, "left", "ID_REGION").sort_values(by = "X'-X/X", ascending=True).reset_index(drop = True)
X_REGION["X'-X/OVERALL(X'-X)"] = (X_REGION["X'-X"]/X_REGION["X'-X"].sum())*100
X_REGION

,ID,ID_REGION,Dane Code,Region,Departamento,X,X',X'-X,X'-X/X,X'-X/OVERALL(X'-X)
0,16,15,50,R16,Meta,56351.707657,34116.015226,-22235.692431,-39.458773,20.570393
1,26,25,85,R26,Casanare,24610.377801,16752.980855,-7857.396946,-31.927169,7.268932
2,27,26,86,R27,Putumayo,6262.363707,4323.745979,-1938.617728,-30.956645,1.793429
3,14,13,44,R14,La Guajira,14410.138626,11224.891478,-3185.247148,-22.104209,2.946694
4,25,24,81,R25,Arauca,8628.173736,6725.994708,-1902.179028,-22.046137,1.759719
5,4,3,13,R4,Bolívar,56380.097781,45917.032476,-10463.065306,-18.558083,9.679454
6,9,8,20,R9,Cesar,24668.781119,20112.999140,-4555.781979,-18.467803,4.214585
7,21,20,68,R21,Santander,91388.115872,79611.697255,-11776.418617,-12.886160,10.894446
8,5,4,15,R5,Boyacá,41046.084819,36510.535559,-4535.549260,-11.049895,4.195868
9,13,12,41,R13,Huila,25505.719588,23237.719008,-2268.000580,-8.892125,2.098143


Caida sectorial 

In [52]:
X_SECTOR = pd.DataFrame(zip(sectores_ix_chained,X, XC), columns = ["ID_SECTOR", "X", "X'"])
X_SECTOR = X_SECTOR.groupby("ID_SECTOR").sum().reset_index()
X_SECTOR["X'-X"] = X_SECTOR["X'"]-X_SECTOR["X"]
X_SECTOR["X'-X/X"] = (X_SECTOR["X'-X"]/X_SECTOR["X"])*100
X_SECTOR = pd.merge(sectores, X_SECTOR, "left", "ID_SECTOR").sort_values(by = "X'-X/X", ascending=True)
X_SECTOR["X'-X/OVERALL(X'-X)"] = (X_SECTOR["X'-X"]/X_SECTOR["X'-X"].sum())*100
X_SECTOR

,ID,ID_SECTOR,Sector,Sector_i,Sector_CIIU,CIIU Rev. 4 A.C.,X,X',X'-X,X'-X/X,X'-X/OVERALL(X'-X)
22,23,22,S23,HC,"Coquización, fabricación de productos de la re...",043 044,45305.000001,15183.655134,-30121.344867,-66.485697,27.865465
6,7,6,S7,HC,Extracción de petróleo crudo y gas natural y a...,018 021 022,61853.000001,23061.162428,-38791.837573,-62.716178,35.886598
33,34,33,S34,HC,Producción de gas; distribución de combustible...,061,7257.000000,3317.613498,-3939.386502,-54.283953,3.644354
5,6,5,S6,HC,Extracción de carbón de piedra y lignito,017,12653.000001,6183.884650,-6469.115351,-51.127127,5.984623
40,41,40,S41,O,Transporte acuático,073,699.000000,581.497554,-117.502446,-16.810078,0.108702
39,40,39,S40,O,Transporte terrestre y transporte por tuberías,072 074,54538.000000,45738.831386,-8799.168614,-16.134014,8.140172
42,43,42,S43,O,Almacenamiento y actividades complementarias a...,076,13203.000000,11690.257624,-1512.742376,-11.457566,1.399448
14,15,14,S15,FP,Elaboración de azúcar y elaboración de panela,030 031,6800.000000,6096.447256,-703.552744,-10.346364,0.650862
38,39,38,S39,O,Mantenimiento y reparación de vehículos automo...,071,11963.000000,11008.719344,-954.280656,-7.976934,0.882812
32,33,32,S33,O,Generación de energía eléctrica; transmisión d...,058 059 060,37590.000000,35479.317200,-2110.682799,-5.615011,1.952607


Caida sectorial agregada

In [53]:
X_SECTOR_I = X_SECTOR.drop(columns=["X'-X/X", "X'-X/OVERALL(X'-X)"]); X_SECTOR_I = X_SECTOR_I.groupby("Sector_i").sum().reset_index()
X_SECTOR_I["X'-X/X"] = (X_SECTOR_I["X'-X"]/X_SECTOR_I["X"])*100
X_SECTOR_I["X'-X/OVERALL(X'-X)"] = (X_SECTOR_I["X'-X"]/X_SECTOR_I["X'-X"].sum())*100
X_SECTOR_I

<ipython-input-53-a05a6d08256d>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_SECTOR_I = X_SECTOR.drop(columns=["X'-X/X", "X'-X/OVERALL(X'-X)"]); X_SECTOR_I = X_SECTOR_I.groupby("Sector_i").sum().reset_index()


,Sector_i,ID,ID_SECTOR,X,X',X'-X,X'-X/X,X'-X/OVERALL(X'-X)
0,A,15,10,7.769300e+04,7.726221e+04,-430.787949,-0.554475,0.398525
1,FP,108,100,8.626200e+04,8.536933e+04,-892.665703,-1.034831,0.825811
2,HC,70,66,1.270680e+05,4.774632e+04,-79321.684293,-62.424595,73.381040
3,O,1247,1211,1.088052e+06,1.060966e+06,-27085.737879,-2.489379,25.057204
4,T,45,44,6.186700e+04,6.150226e+04,-364.736290,-0.589549,0.337420


### Recuperación con Sectores seleccionados

In [55]:
sector_recuperador = "T"

In [56]:
A_SHOCK = RS_DGP[RS_DGP.Sector_i==sector_recuperador].index
CAIDA_XABS = np.abs(XC.sum()-X.sum())

In [57]:
for i in tq(np.arange(0.01, 1, 0.01)):
    
    impulso = i
    crecimiento  = 1+impulso

    YCA = YC.copy()
    ZCA = ZC.copy()

    for i in A_SHOCK:
        ZCA[i,:] = ZCA[i,:]*crecimiento
        ZCA[:,i] = ZCA[:,i]*crecimiento

        YCA[i] = YCA[i]*crecimiento

    # ZC[np.ix_(prueba, prueba)] *= choque
    IC = np.eye(MIP.shape[0])
    IZCA = I-ZCA
    BCA = np.linalg.inv(IZCA)
    XCA = BCA@YCA


    AUMENTO_X = (XCA.sum()-XC.sum())

    if AUMENTO_X>=CAIDA_XABS:
        break
        


print(CAIDA_X)
print(AUMENTO_X)


 59%|█████▊    | 58/99 [00:59<00:42,  1.03s/it]

-108095.61211408558
108237.81128890323


In [61]:
AUMENTO_X/XC.sum()

0.0812080163707589

Aumento Regional

In [62]:
X_REGIONR = pd.DataFrame(zip(regiones_ix_chained,XCA), columns = ["ID_REGION","X''"])
X_REGIONR = X_REGIONR.groupby("ID_REGION").sum().reset_index()
X_REGIONR = pd.merge(regiones, X_REGIONR, "left", "ID_REGION")[["ID_REGION", "X''"]]
X_REGIONBR  = pd.merge(X_REGION, X_REGIONR, "left", "ID_REGION")
X_REGIONBR["X''-X'"] = X_REGIONBR["X''"]-X_REGIONBR["X'"]
X_REGIONBR["X''-X'/X'"] = (X_REGIONBR["X''-X'"]/X_REGIONBR["X'"])*100
X_REGIONBR["X''-X'/OVERALL(X'''-X)"] = (X_REGIONBR["X''-X'"]/X_REGIONBR["X''-X'"].sum())*100
X_REGIONBR

,ID,ID_REGION,Dane Code,Region,Departamento,X,X',X'-X,X'-X/X,X'-X/OVERALL(X'-X),X'',X''-X',X''-X'/X',X''-X'/OVERALL(X'''-X)
0,16,15,50,R16,Meta,56351.707657,34116.015226,-22235.692431,-39.458773,20.570393,36561.789906,2445.774680,7.168993,2.259631
1,26,25,85,R26,Casanare,24610.377801,16752.980855,-7857.396946,-31.927169,7.268932,18383.598598,1630.617743,9.733299,1.506514
2,27,26,86,R27,Putumayo,6262.363707,4323.745979,-1938.617728,-30.956645,1.793429,4967.723818,643.977839,14.893979,0.594966
3,14,13,44,R14,La Guajira,14410.138626,11224.891478,-3185.247148,-22.104209,2.946694,12357.288705,1132.397227,10.088269,1.046212
4,25,24,81,R25,Arauca,8628.173736,6725.994708,-1902.179028,-22.046137,1.759719,7528.375636,802.380928,11.929550,0.741313
5,4,3,13,R4,Bolívar,56380.097781,45917.032476,-10463.065306,-18.558083,9.679454,49876.404076,3959.371600,8.622882,3.658030
6,9,8,20,R9,Cesar,24668.781119,20112.999140,-4555.781979,-18.467803,4.214585,21880.585472,1767.586332,8.788278,1.633058
7,21,20,68,R21,Santander,91388.115872,79611.697255,-11776.418617,-12.886160,10.894446,83547.251627,3935.554372,4.943437,3.636025
8,5,4,15,R5,Boyacá,41046.084819,36510.535559,-4535.549260,-11.049895,4.195868,39197.019785,2686.484226,7.358107,2.482020
9,13,12,41,R13,Huila,25505.719588,23237.719008,-2268.000580,-8.892125,2.098143,24731.684435,1493.965427,6.429054,1.380262


In [63]:
# Baja en producción de HC -> Aumento en sector X -> Si sector X está concentrado en regiones más ricas entonces habremos recuperado la economía pero con una mayor desigualdad en el agregado

In [65]:
# X_REGIONBR.to_excel("prueba.xlsx")

¿Podemos interiorizar lo anterior en el análisis?

Aumento sectorial 

In [66]:
X_SECTORR = pd.DataFrame(zip(sectores_ix_chained,XCA), columns = ["ID_SECTOR", "X''"])
X_SECTORR = X_SECTORR.groupby("ID_SECTOR").sum().reset_index()
X_SECTORR = pd.merge(sectores, X_SECTORR, "left", "ID_SECTOR")[["ID_SECTOR", "X''"]]
X_SECTORBR  = pd.merge(X_SECTOR, X_SECTORR, "left", "ID_SECTOR")
X_SECTORBR["X''-X'"] = X_SECTORBR["X''"]-X_SECTORBR["X'"]
X_SECTORBR["X''-X'/X'"] = (X_SECTORBR["X''-X'"]/X_SECTORBR["X'"])*100
X_SECTORBR["X''-X'/OVERALL(X''-X')"] = (X_SECTORBR["X''-X'"]/X_SECTORBR["X''-X'"].sum())*100
X_SECTORBR

,ID,ID_SECTOR,Sector,Sector_i,Sector_CIIU,CIIU Rev. 4 A.C.,X,X',X'-X,X'-X/X,X'-X/OVERALL(X'-X),X'',X''-X',X''-X'/X',X''-X'/OVERALL(X''-X')
0,23,22,S23,HC,"Coquización, fabricación de productos de la re...",043 044,45305.000001,15183.655134,-30121.344867,-66.485697,27.865465,15709.429038,525.773904,3.462762,0.485758
1,7,6,S7,HC,Extracción de petróleo crudo y gas natural y a...,018 021 022,61853.000001,23061.162428,-38791.837573,-62.716178,35.886598,23134.159959,72.997531,0.316539,0.067442
2,34,33,S34,HC,Producción de gas; distribución de combustible...,061,7257.000000,3317.613498,-3939.386502,-54.283953,3.644354,3460.031435,142.417937,4.292783,0.131579
3,6,5,S6,HC,Extracción de carbón de piedra y lignito,017,12653.000001,6183.884650,-6469.115351,-51.127127,5.984623,6196.822705,12.938055,0.209222,0.011953
4,41,40,S41,O,Transporte acuático,073,699.000000,581.497554,-117.502446,-16.810078,0.108702,604.986695,23.489141,4.039422,0.021701
5,40,39,S40,O,Transporte terrestre y transporte por tuberías,072 074,54538.000000,45738.831386,-8799.168614,-16.134014,8.140172,49227.679085,3488.847700,7.627759,3.223317
6,43,42,S43,O,Almacenamiento y actividades complementarias a...,076,13203.000000,11690.257624,-1512.742376,-11.457566,1.399448,12380.183548,689.925924,5.901717,0.637417
7,15,14,S15,FP,Elaboración de azúcar y elaboración de panela,030 031,6800.000000,6096.447256,-703.552744,-10.346364,0.650862,6814.593917,718.146661,11.779757,0.663490
8,39,38,S39,O,Mantenimiento y reparación de vehículos automo...,071,11963.000000,11008.719344,-954.280656,-7.976934,0.882812,11391.625877,382.906533,3.478211,0.353764
9,33,32,S33,O,Generación de energía eléctrica; transmisión d...,058 059 060,37590.000000,35479.317200,-2110.682799,-5.615011,1.952607,36689.687299,1210.370099,3.411481,1.118251


In [67]:
# X_SECTORBR.to_excel("prueba.xlsx")

Aumento sectorial agregada

In [68]:
X_SECTOR_IBR = X_SECTORBR.drop(columns=["X'-X/X", "X'-X/OVERALL(X'-X)", "X''-X'/OVERALL(X''-X')"]); X_SECTOR_IBR = X_SECTOR_IBR.groupby("Sector_i").sum().reset_index()
X_SECTOR_IBR["X'-X/X"] = (X_SECTOR_IBR["X'-X"]/X_SECTOR_IBR["X"])*100
X_SECTOR_IBR["X'-X/OVERALL(X'-X)"] = (X_SECTOR_IBR["X'-X/X"]/X_SECTOR_IBR["X'-X/X"].sum())*100
X_SECTOR_IBR["X''-X'/X'"] = (X_SECTOR_IBR["X''-X'"]/X_SECTOR_IBR["X'"])*100
X_SECTOR_IBR["X''-X'/OVERALL(X''-X')"] = (X_SECTOR_IBR["X''-X'/X'"]/X_SECTOR_IBR["X''-X'/X'"].sum())*100
X_SECTOR_IBR = X_SECTOR_IBR[["Sector_i", "ID","X", "X'","X'-X", "X'-X/X", "X'-X/OVERALL(X'-X)", "X''", "X''-X'", "X''-X'/X'","X''-X'/OVERALL(X''-X')"]]
X_SECTOR_IBR

<ipython-input-68-671ddb2d0a9d>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  X_SECTOR_IBR = X_SECTORBR.drop(columns=["X'-X/X", "X'-X/OVERALL(X'-X)", "X''-X'/OVERALL(X''-X')"]); X_SECTOR_IBR = X_SECTOR_IBR.groupby("Sector_i").sum().reset_index()


,Sector_i,ID,X,X',X'-X,X'-X/X,X'-X/OVERALL(X'-X),X'',X''-X',X''-X'/X',X''-X'/OVERALL(X''-X')
0,A,15,7.769300e+04,7.726221e+04,-430.787949,-0.554475,0.826429,8.889814e+04,11635.930819,15.060313,14.875830
1,FP,108,8.626200e+04,8.536933e+04,-892.665703,-1.034831,1.542387,9.945858e+04,14089.240836,16.503866,16.301700
2,HC,70,1.270680e+05,4.774632e+04,-79321.684293,-62.424595,93.042128,4.850044e+04,754.127427,1.579446,1.560099
3,O,1247,1.088052e+06,1.060966e+06,-27085.737879,-2.489379,3.710350,1.103298e+06,42331.483987,3.989899,3.941024
4,T,45,6.186700e+04,6.150226e+04,-364.736290,-0.589549,0.878706,1.009293e+05,39427.028220,64.106629,63.321347


In [69]:
# X_SECTOR_IBR.to_excel("prueba.xlsx")

In [70]:
# with pd.ExcelWriter('ESCENARIO RECUPERACION AGRICULTURA.xlsx') as writer:
# with pd.ExcelWriter('ESCENARIO RECUPERACION TURISMO.xlsx') as writer:
    X_SECTORBR.to_excel(writer, sheet_name='X_SECTORBR', index = False)
    X_SECTOR_IBR.to_excel(writer, sheet_name='X_SECTOR_IBR', index = False)
    X_REGIONBR.to_excel(writer, sheet_name='X_REGIONBR', index = False)

<ipython-input-70-42dc78ed7a51>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  with pd.ExcelWriter('ESCENARIO RECUPERACION TURISMO.xlsx') as writer:
